In [1]:
#! pip install sktime
#! pip install standard-precip
import pandas as pd
import numpy as np

from datetime import datetime
import glob
import xarray as xr
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm

from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing 

from sklearn.linear_model import LinearRegression

from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.utils.plotting import plot_series

from scipy.stats import gamma

from standard_precip.spi import SPI
from standard_precip.utils import plot_index

import seaborn as sns

In [2]:
#read dataframe 
# canola_2 = df = pd.read_csv('/kaggle/input/rm-yields-data/rm-yields-data.csv', header=0, index_col=0, parse_dates=True)
# canola_small = canola_2.iloc[:, [0, 2]].copy()

#read dataframe 
canola_2 = df = pd.read_csv('../data/rm-yields-data.csv', header=0, index_col=0, parse_dates=True)
canola_small = canola_2.iloc[:, [0, 2]].copy()

In [3]:
start_year = 1938
start_analysis = 1990
exclude_years = start_analysis - start_year
#cut 70s and 80s as well 
#cut of first 52 observations (NAs)
canola_small.drop(canola_small.index[:exclude_years], inplace=True)

#filter out every observation that contains NAs
canola_filtered = canola_small.groupby('RM').filter(lambda group: not group['Canola'].isnull().any())

# how may districts? 148
num_districts = canola_filtered.groupby('RM').ngroups
print(num_districts)
#excluding 70s and 80s lead to 36 more colmplete districts 

184


In [4]:
# Group by 'RM' and check if 'Canola' has any missing values in each group
districts_with_full_data = canola_filtered.groupby('RM')['Canola'].apply(lambda group: not group.isnull().any())

# Extract the list of districts with full data
districts_with_full_data_list = districts_with_full_data[districts_with_full_data].index.tolist()

In [5]:
# select weather data
#open only the years from 1990 til 2022

# Define the directory path and pattern for the NetCDF files
# directory_path = '/kaggle/input/copernicus-data/'
# file_pattern = '*.nc'
directory_path = '../data/all_raw_data/'
file_pattern = 'data_*.nc'


# Get a list of files matching the pattern
files_to_open = glob.glob(os.path.join(directory_path, file_pattern))

# Open only the files for the years 1990 to 2022
years_to_open = list(map(str, range(start_analysis, 2023)))
files_to_open = [file for file in files_to_open if any(year in file for year in years_to_open)]

# Use open_mfdataset to open the selected files
cop_all_90 = xr.open_mfdataset(files_to_open, combine='by_coords')


# open data from 1971 til 1989 as training data 

training_years_to_open = list(map(str, range(1971, 1989)))
training_files_to_open = [file for file in files_to_open if any(year in file for year in years_to_open)]

# Use open_mfdataset to open the selected files
training_data = xr.open_mfdataset(training_files_to_open, combine='by_coords')

In [6]:
print(training_data)

<xarray.Dataset>
Dimensions:    (longitude: 88, latitude: 41, time: 169488)
Coordinates:
  * longitude  (longitude) float32 -110.0 -109.9 -109.8 ... -101.5 -101.4 -101.3
  * latitude   (latitude) float32 53.0 52.9 52.8 52.7 ... 49.3 49.2 49.1 49.0
  * time       (time) datetime64[ns] 1990-04-01 ... 2022-10-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(5136, 41, 88), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(5136, 41, 88), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-01-23 18:32:47 GMT by grib_to_netcdf-2.24.0: /opt/ecmw...


In [7]:
# # center points for regions
# df_regions = pd.read_csv('/kaggle/input/cgn-sk-csv-eng/cgn_sk_csv_eng.csv')
# df_rms = df_regions[['Geographical Name','Latitude', 'Longitude']][df_regions['Generic Term'] == 'Rural Municipality']
# df_rms['region_index'] = df_rms['Geographical Name'].str.split(' ').str[-1].astype(int)

# center points for regions
df_regions = pd.read_csv(r'../data/cgn_sk_csv_eng.csv')
df_rms = df_regions[['Geographical Name','Latitude', 'Longitude']][df_regions['Generic Term'] == 'Rural Municipality']
df_rms['region_index'] = df_rms['Geographical Name'].str.split(' ').str[-1].astype(int)

In [8]:
#print(df_rms)

In [9]:
def get_center(region):
    avg_lat = df_rms['Latitude'][df_rms['region_index'] == region].item()
    avg_long = df_rms['Longitude'][df_rms['region_index'] == region].item()
    return avg_lat, avg_long

def detrend_ts(df_region):
    # linear detrending
    forecaster = PolynomialTrendForecaster(degree=2)
    transformer = Detrender(forecaster=forecaster)
    yt = transformer.fit_transform(df_region['Canola'])
    return yt

In [10]:
def merge_canola_weather_data(region = 310):
    # select data from region with center point
    center_lat, center_long = get_center(region)
    cropped_data_tmp = cop_all_90.sel(longitude=center_long, latitude=center_lat,method='nearest')
    cropped_data_tmp_train = training_data.sel(longitude=center_long, latitude=center_lat,method='nearest')
    
    # get residuals for canola yield
    df_tmp = canola_filtered[canola_filtered['RM'] == region]
    residuals = detrend_ts(df_tmp)

    # merge weather data and canola residuals
    df_weather_region = cropped_data_tmp.to_dataframe()
    df_weather_region_train = cropped_data_tmp_train.to_dataframe()

    df_weather_region['region'] = region

    column_to_append = residuals.tolist()
    years = df_weather_region.index.year
    df_weather_region['Canola_detrended'] = [column_to_append[year - start_analysis] for year in years]
    df_weather_region.drop(['longitude','latitude'],axis=1,inplace=True)
    
    df_weather_region_train['region'] = region

    column_to_append = residuals.tolist()
    years = df_weather_region_train.index.year
    df_weather_region_train['Canola_detrended'] = [column_to_append[year - start_analysis] for year in years]
    df_weather_region_train.drop(['longitude','latitude'],axis=1,inplace=True)

    return df_weather_region, pd.DataFrame(residuals), df_weather_region_train

In [11]:
def calculate_spi(prcp_data, scale=1):
    # Step 1: Calculate L-moments
    n = len(prcp_data)
    prcp_data_sorted = np.sort(prcp_data)
    
    # L-moment ratio
    l_moment_1 = np.sum(prcp_data_sorted) / n
    l_moment_2 = np.sum((2 * np.arange(1, n + 1) - 1 - n) * prcp_data_sorted) / (n ** 2)
    
    # Step 2: Estimate parameters of gamma distribution
    k = l_moment_1 / l_moment_2
    theta = l_moment_2 / k
    
    # Step 3: Calculate SPI values
    spi_values = gamma.ppf((np.arange(1, n + 1) - 0.35) / (n + 0.3), a=k, scale=theta * scale)
    
    return spi_values

In [12]:
def calc_temp_features(df_weather_region, df_year):
    for month in range(4,11):    
        daily_max_temperatures = df_weather_region.resample('D').max()
        monthly_avg_max_temperatures = daily_max_temperatures.resample('MS').mean()
        
    #     dist1_df_month = dist1_df.resample('MS').mean()
        month_data = monthly_avg_max_temperatures[monthly_avg_max_temperatures.index.month == month]
        column_to_append = month_data['t2m'].tolist()
        df_year.loc[:, f'average_max_temp_in_{month}'] = column_to_append
    return df_year

In [13]:
def calc_spi_features(df_weather_region, df_year):
    for month in range(4, 11):
        # tried resampling in various ways but none worked
        tp_in_month = df_weather_region[df_weather_region.index.month == month]

        spi = SPI()

        # Assuming spi.calculate is the SPI calculation function
        spi_values = spi.calculate(
            tp_in_month.reset_index(),
            'time',
            'tp',
            freq="M",
            scale=1,
            fit_type="lmom",
            dist_type="gam"
        )
        
        # may have to aggregate here somehow

        # Add each SPI column separately
        for col_name in spi_values.columns:
            df_year[f'SPI_in_{month}_{col_name}'] = spi_values[col_name]
            
    return df_year

In [14]:
def calc_summer_days(df_weather_region, df_year):

    testing_data = df_weather_region['t2m'].resample('D').max()
    testing_data = testing_data.dropna()
    test_df_summer = testing_data.to_frame(name='t2m')
    test_df_summer['month_day'] = test_df_summer.index.strftime('%m-%d')
    test_df_summer['above_25'] = test_df_summer['t2m'] > 298
    days_over_25 = test_df_summer.groupby(test_df_summer.index.year)['above_25'].apply(sum)
    column_to_append = days_over_25.tolist()
    df_year.loc[:, f'days_above_25'] = column_to_append
    
    return df_year

In [15]:
def calc_frost_days(df_weather_region, df_year):

    testing_data = df_weather_region['t2m'].resample('D').max()
    testing_data = testing_data.dropna()
    test_df_frost = testing_data.to_frame(name='t2m')
    test_df_frost['month_day'] = test_df_frost.index.strftime('%m-%d')
    test_df_frost['under_0'] = test_df_frost['t2m'] < 273
    days_under_0 = test_df_frost.groupby(test_df_frost.index.year)['under_0'].apply(sum)
    column_to_append = days_under_0.tolist()
    df_year.loc[:, f'days_under_0'] = column_to_append
    
    return df_year

In [16]:
# function to calculate the longest consecutive true streak

def longest_consecutive_true_streak(series):
    
    # Convert the series to integers (True to 1, False to 0) for easier streak calculation
    as_ints = series.astype(int)
    # Calculate the difference to identify changes in streaks
    diff = as_ints.diff()
    # Start a new group every time there's a change from 0 to 1 (start of a new streak)
    groups = (diff == 1).cumsum()
    # Use the groups to isolate consecutive trues, then count them, keeping the max
    streak_lengths = as_ints.groupby(groups).sum()
    # Return the length of the longest streak
    return streak_lengths.max()

In [17]:
def longest_dry_spell(df_weather_region, df_year):
    
    dist1_df_perci = df_weather_region.resample('D').sum()
    dist1_df_perci_wo0 = dist1_df_perci[dist1_df_perci['tp'] != 0].copy()
    
    # Threshhold is 1mm or 0.001m, here precipitation is likely measured in m 
    
    dist1_df_perci_wo0.loc[:,'less_than_0.001'] = dist1_df_perci_wo0['tp'] < 0.001
    longest_dry_spell_per_year = dist1_df_perci_wo0.groupby(dist1_df_perci_wo0.index.year)['less_than_0.001'].apply(longest_consecutive_true_streak)
    
    column_to_append = longest_dry_spell_per_year.tolist()
    df_year.loc[:, f'longest_dry_spell'] = column_to_append
    
    return df_year

In [18]:
def longest_wet_spell(df_weather_region, df_year):
    
    dist1_df_perci = df_weather_region.resample('D').sum()
    dist1_df_perci_wo0 = dist1_df_perci[dist1_df_perci['tp'] != 0].copy()
    
    # Threshhold is 1mm or 0.001m, here precipitation is likely measured in m 
    
    dist1_df_perci_wo0.loc[:,'more_than_0.001'] = dist1_df_perci_wo0['tp'] > 0.001
    longest_wet_spell_per_year = dist1_df_perci_wo0.groupby(dist1_df_perci_wo0.index.year)['more_than_0.001'].apply(longest_consecutive_true_streak)
    
    column_to_append = longest_wet_spell_per_year.tolist()
    df_year.loc[:, f'longest_wet_spell'] = column_to_append
    
    return df_year

In [19]:
def over_95_precipitation(df_weather_region, df_weather_region_train, df_year):
    
    dist1_df_perci = df_weather_region.resample('D').sum()
    dist1_df_perci_wo0 = dist1_df_perci[dist1_df_perci['tp'] != 0]
    testing_data_pre = dist1_df_perci_wo0["tp"]
    
    dist1_df_perci_train = df_weather_region_train.resample('D').sum()
    dist1_df_perci_wo0_train = dist1_df_perci_train[dist1_df_perci_train['tp'] != 0]
    training_data_pre = dist1_df_perci_wo0_train["tp"]


    # calculate for every day the 90% quantile
    quantile_95_series = training_data_pre.groupby([training_data_pre.index.month, training_data_pre.index.day]).quantile(0.95)
    quantile_95_series.index = quantile_95_series.index.map(lambda x: f"{x[0]:02d}-{x[1]:02d}")

    test_df_pre = testing_data_pre.to_frame(name='tp')
    test_df_pre['month_day'] = test_df_pre.index.strftime('%m-%d')

    # map the 90th percentile values from quantile_90_series to the test series
    test_df_pre['quantile_95'] = test_df_pre['month_day'].apply(lambda x: quantile_95_series.get(x, pd.NA))

    # compare each test value to its corresponding 90th percentile value
    test_df_pre['over_quantile_95'] = test_df_pre['tp'] > test_df_pre['quantile_95']
    test_df_pre.drop(['month_day', 'quantile_95'], axis=1, inplace=True)

    # Group the DataFrame by year, and apply the function to find the longest streak of True values
    days_over_95 = test_df_pre.groupby(test_df_pre.index.year)['over_quantile_95'].apply(sum)

    column_to_append = days_over_95.tolist()
    df_year.loc[:, f'days_over_95_precipitation'] = column_to_append
    
    return df_year

In [20]:
def heat_wave(df_weather_region, df_weather_region_train, df_year):

    training_data = df_weather_region_train['t2m'].resample('D').max()
    training_data = training_data.dropna()

    testing_data = df_weather_region['t2m'].resample('D').max()
    testing_data = testing_data.dropna()

    # calculate for every day the 90% quantile
    quantile_90_series = training_data.groupby([training_data.index.month, training_data.index.day]).quantile(0.9)
    quantile_90_series.index = quantile_90_series.index.map(lambda x: f"{x[0]:02d}-{x[1]:02d}")

    test_df = testing_data.to_frame(name='value')
    test_df['month_day'] = test_df.index.strftime('%m-%d')

    # map the 90th percentile values from quantile_90_series to the test series
    test_df['quantile_90'] = test_df['month_day'].apply(lambda x: quantile_90_series.get(x, pd.NA))

    # compare each test value to its corresponding 90th percentile value
    test_df['is_above_quantile_90'] = test_df['value'] > test_df['quantile_90']
    test_df.drop(['month_day', 'quantile_90'], axis=1, inplace=True)

    # Group the DataFrame by year, and apply the function to find the longest streak of True values
    longest_heat_streak_by_year = test_df.groupby(test_df.index.year)['is_above_quantile_90'].apply(longest_consecutive_true_streak)

    column_to_append = longest_heat_streak_by_year.tolist()
    df_year.loc[:, f'longest_heat_wave'] = column_to_append
    
    return df_year

In [21]:
def cold_wave(df_weather_region, df_weather_region_train, df_year):
    
    training_data = df_weather_region_train['t2m'].resample('D').min()
    training_data = training_data.dropna()

    testing_data = df_weather_region['t2m'].resample('D').min()
    testing_data = testing_data.dropna()
    
    # calculate for every day the 90% quantile
    quantile_10_series = training_data.groupby([training_data.index.month, training_data.index.day]).quantile(0.1)
    quantile_10_series.index = quantile_10_series.index.map(lambda x: f"{x[0]:02d}-{x[1]:02d}")

    test_df_cold = testing_data.to_frame(name='value')
    test_df_cold['month_day'] = test_df_cold.index.strftime('%m-%d')

    # map the 90th percentile values from quantile_90_series to the test series
    test_df_cold['quantile_10'] = test_df_cold['month_day'].apply(lambda x: quantile_10_series.get(x, pd.NA))

    # compare each test value to its corresponding 90th percentile value
    test_df_cold['is_under_quantile_10'] = test_df_cold['value'] < test_df_cold['quantile_10']
    test_df_cold.drop(['month_day', 'quantile_10'], axis=1, inplace=True)

    # Group the DataFrame by year, and apply the function to find the longest streak of True values
    longest_streak_by_year_cold = test_df_cold.groupby(test_df_cold.index.year)['is_under_quantile_10'].apply(longest_consecutive_true_streak)

    column_to_append = longest_streak_by_year_cold.tolist()
    df_year.loc[:, f'longest_cold_wave'] = column_to_append
    
    return df_year

In [22]:
available_regions = [region for region in districts_with_full_data_list if region in df_rms['region_index'].to_list()]

In [23]:
available_regions.remove(278)
available_regions.remove(529)

In [24]:
dfs_of_years = []
for region in available_regions:
    print(region)
    df_weather_region, df_year, df_weather_region_train  = merge_canola_weather_data(region)
    df_year.index = df_year.index.year
    df_year['region'] = region
    df_year = calc_temp_features(df_weather_region, df_year)
    df_year = calc_spi_features(df_weather_region, df_year)
    df_year = calc_summer_days(df_weather_region,df_year)
    df_year = calc_frost_days(df_weather_region,df_year)
    df_year = longest_dry_spell(df_weather_region,df_year)
    df_year = longest_wet_spell(df_weather_region,df_year)
    df_year = over_95_precipitation(df_weather_region, df_weather_region_train, df_year)
    df_year = heat_wave(df_weather_region, df_weather_region_train, df_year)
    df_year = cold_wave(df_weather_region, df_weather_region_train, df_year)
    dfs_of_years.append(df_year)

1
2
3
31
32
33
34
61
63
64
65
66
91
92
93
95
96
121
122
123
124
125
126
127
131
151
152
153
154
155
156
157
158
181
183
184
185
186
189
190
194
211
213
214
216
217
218
219
220
221
222
223
224
225
241
243
244
245
246
247
248
250
251
252
253
254
255
256
271
273
276
277
280
281
282
283
284
288
304
305
307
308
309
310
312
313
314
315
316
317
320
331
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
349
350
351
352
366
367
368
369
370
371
372
373
376
377
378
379
380
381
382
394
395
397
398
399
400
401
402
403
404
405
406
409
410
411
426
427
428
429
430
431
435
436
437
438
439
440
442
456
457
458
459
460
461
463
464
466
467
468
471
472
486
487
488
490
491
493
494
497
499
501
502
520
588
622


In [25]:
df_full = pd.concat(dfs_of_years)

In [26]:
df_full

,Canola,region,average_max_temp_in_4,average_max_temp_in_5,average_max_temp_in_6,average_max_temp_in_7,average_max_temp_in_8,average_max_temp_in_9,average_max_temp_in_10,SPI_in_4_time,...,SPI_in_10_time,SPI_in_10_tp,SPI_in_10_tp_calculated_index,days_above_25,days_under_0,longest_dry_spell,longest_wet_spell,days_over_95_precipitation,longest_heat_wave,longest_cold_wave
Year,,,,,,,,,,,,,,,,,,,,,
1990,0.127132,1,284.965759,290.526489,297.082458,299.412781,300.752075,296.791382,286.036377,1992-04-23 22:00:00,...,1992-10-21 22:00:00,-1.862645e-09,-0.380289,72,0,7,11,11,7,5
1991,2.520378,1,287.648346,291.984650,296.854401,297.652344,299.938751,292.543549,282.321960,1992-04-23 23:00:00,...,1992-10-21 23:00:00,-1.862645e-09,-0.380289,51,4,6,13,18,6,4
1992,-6.339489,1,283.242584,293.701874,295.445709,294.548126,297.033264,292.229004,285.503967,1992-04-24 00:00:00,...,1992-10-22 00:00:00,-1.862645e-09,-0.380289,37,2,7,11,8,5,7
1993,4.147971,1,284.697113,292.762695,293.425385,294.037415,296.177429,291.031281,284.688232,1992-04-24 01:00:00,...,1992-10-22 01:00:00,-1.862645e-09,-0.380289,26,1,6,14,10,5,5
1994,2.081733,1,285.144440,293.432678,294.774536,297.061951,297.072021,294.903839,286.052094,1992-04-24 02:00:00,...,1992-10-22 02:00:00,-1.862645e-09,-0.380289,38,0,9,15,16,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,-0.642088,622,277.338013,294.617126,296.039825,297.508179,297.946716,285.104767,280.153503,1992-04-25 02:00:00,...,1992-10-23 02:00:00,-1.862645e-09,-0.389745,51,11,9,20,13,4,9
2019,5.176214,622,283.511230,289.193604,293.939178,295.673492,293.541107,289.933136,280.083069,1992-04-25 03:00:00,...,1992-10-23 03:00:00,-1.862645e-09,-0.389745,11,3,7,33,13,3,4
2020,-1.002221,622,277.344818,290.024719,293.347931,296.085846,296.669983,291.383881,279.867188,1992-04-25 04:00:00,...,1992-10-23 04:00:00,-1.862645e-09,-0.389745,23,17,6,10,9,3,11
